# CNN (Convolutional Neural Network)

- 참고
    - Image classification with deep convolutional neural networks (Khan et al., 2020)
    - ImageNet Classification with Deep Convolutional Neural Networks (Krizhevsky et al., 2012)

- Convolutional Neural Network(CNN)은 컴퓨터 비전 및 이미지 처리와 관련된 여러 대회에서 우수한 성능을 보인 특별한 유형의 신경망
- CNN은 이미지 내의 특징을 추출하고 이를 이용해 이미지를 분류하거나 객체를 검출하는 등의 작업을 수행할 수 있음
- CNN은 입력 이미지를 여러 개의 레이어로 처리하며, 각 레이어에서는 입력 이미지와 필터를 합성곱하여 특징 맵(feature map)을 생성
- 이때 필터는 이미지에서 특정한 패턴을 찾아내기 위한 가중치 값으로, 학습 과정에서 자동으로 조정됨
- 레이어를 여러 번 거침으로써, 점점 더 추상적인 특징을 추출할 수 있음

- ImageNetClassification with Deep Convolutional Neural Networks에서는 8개의 레이어로 이루어진 CNN 아키텍처인 AlexNet을 제안함
- 이 아키텍처는 풀링 레이어, 컨볼루션 레이어, 활성화 함수, 드롭아웃 레이어 등 다양한 컴포넌트로 구성됨
- 또한, 큰 데이터셋과 병렬 컴퓨팅을 이용하여 CNN의 학습을 가속화하는 방법도 제안하고 있음

- AlexNet은 기존의 신경망 아키텍처에 비해 몇 가지 차이점이 있음
    1. 더 많은 학습 데이터 사용: AlexNet은 120만 개 이상의 이미지를 사용하여 학습
    2. GPU를 사용한 병렬 처리: AlexNet은 2개의 GPU를 사용하여 효율적으로 학습할 수 있도록 설계
    3. ReLU 활성화 함수 사용: 기존의 신경망에서는 sigmoid 함수나 tanh 함수를 사용했지만, AlexNet에서는 ReLU(Rectified Linear Unit) 함수를 사용하여 학습 속도를 높임
    4. Local Response Normalization: 이전 레이어에서 활성화된 뉴런들이 다음 레이어에서 높은 활성화 값을 가지도록 정규화를 적용
    5. Dropout: 학습 데이터에서 무작위로 뉴런을 제거하면서 학습하는 방법으로, 오버피팅(overfitting)을 방지하는 데 효과적
- 논문에서는 또한 maxpooling과 conv2d도 자세히 설명하고 있음
    - Maxpooling은 이미지의 크기를 줄이고, 불필요한 정보를 제거하는 데 사용됨
    - Conv2d는 이미지에서 특징을 추출하는 데 사용되는 컨볼루션 필터를 적용하는 방법

<img src = "./image/CNN.png">

- 합성곱 : 입력 데이터에서 유용한 특성만 드러나게 하는 것
    - 합성곱 계산을 통해 만들어진 출력을 특성맵(feature map)이라고 부름
    
- 일반적으로 1개 이상의 합성곱 층을 사용한 인공 신경망을 합성곱 신경망이라고 부름
    - 합성곱 층만을 이용한 신경망은 아님
    
- 합성곱과 밀집층의 차이
    - 밀집층에서는 뉴런마다 입력 개수만큼의 가중치가 존재
    - 합성곱은 입력 데이터의 일부에 가중치를 곱함
    - 합성곱 신경망에서는 유닛을 필터(filter) 또는 커널(kernel)이라고 부름
    
- 입력데이터가 2차원 배열이라면 커널도 2차원이어야함
- 위 이미지에서 커널 크기는 (3, 3)
- 밀집층에서 여러 개의 뉴런을 사용하듯 합성곱 층에서도 여려 개의 필터를 사용할 수 있음
- 합성곱은 2차원의 형태를 유지함
    -  공간적 특성의 손실을 줄일 수 있기 때문에 이미지 처리 분야에서 성능이 뛰어남
    
    
- 합성곱 연산 순서
    1. 왼쪽 위 모서리부터 합성곱을 시작
    2. 1개의 출력을 계산
    3. 오른쪽으로 이동
        - 오른쪽으로 이동 할 수 없으면 아래로 이동
    4. 2 ~ 3의 과정을 반복해 합성곱 연산을 수행

In [1]:
from tensorflow import keras

In [2]:
# 합성곱
keras.layers.Conv2D(10,
                    kernel_size = (3, 3),
                    activation = "relu",
                    padding = "same",
                    strides = 1)

- Conv2D의 첫 번째 매개변수는 필터의 개수(필수)
- kernel_size = 필터의 크기(필수)
    - 일반적으로 (3, 3)이나 (5, 5)의 크기로 사용
- 활성화 함수
    - 합성곱 신경망에서 특성맵은 절편과 활성화 함수를 적용한 후의 결과물

## 패딩

<img src = "./image/padding.png">

- 입력 배열의 주위를 가상의 원소로 채우는 것
- 합성곱층을 통과하면 출력의 크기가 입력데이터의 크기보다 작아지게 되는데 이것을 방지하기 위해 사용
    - 마치 (4, 4)보다 더 큰 입력데이터가 들어온 것 처럼 계산
    - (6, 6)의 데이터를 (3, 3)크기의 커널로 합성곱 연산을 하면 출력의 크기가 (4, 4)로 유지됨
    
- 실제로는 입력값이 아니기 때문에 패딩은 0으로 채움
    - 값이 0으로 채워져 있기 때문에 계산에 영향을 미치지 않음
    
    
- 세임 패딩(same padding) : 입력과 특성맵의 크기를 동일하게 만들기 위해 입력 데이터 주위에 0으로 패딩하는 것
    - 일반적으로는 세임 패딩이 많이 사용됨
    
- 밸리드 패딩(valid padding) : 패딩 없이 순수한 입력배열에서만 합성곱을 해 특성맵을 만드는 것
    - 특성 맵의 크기가 입력보다 줄어듦
    

- 패딩을 사용하는 이유
    - 패딩을 사용하지 않으면 입력값의 가운데에 있는 원소와 모서리 부분의 사용 비율이 크게 차이남
    - 적절하게 패딩을 사용하면 이미지 주변의 정보 소실을 막을 수 있음

## 스트라이드

<img src = "./image/stride.png">

- 필터를 적용하는 위치의 간격(이동의 크기)
- 기본값은 1
    - 오른쪽으로 이동하는 크기와 아래쪽으로 이동하는 크기를 (1, 1)과 같이 튜플로 각각 지정할 수 있음
    - 일반적으로 가로세로의 크기를 똑같이 지정
    - 1보다 큰 스트라이드를 사용하는 경우도 드문 편이었으나
        - 최근에는 maxpooling을 사용하지 않고 스트라이드를 높이는 방식이 점점 도입됨

## 풀링

<img src = "./image/pooling.png">

- 합성곱 층에서 만든 특성앱의 가로세로 크기를 줄이는 역할을 수행
    - 특성맵의 크기는 변하지 않음
    
- 합성곱 층에서 추출한 특징을 유지하면서 계산량을 줄여주고 다음 계층 신경망과 쉽게 연결해주기 위해서 사용
- 풀링에는 가중치가 없고 최댓값을 계산하거나(max pooling) 평균값을 계산(average pooling)
    - average pooling보다는 max pooling이 많이 사용됨
    - average pooling은 특성맵의 중요한 정보를 평균 계산하는 과정에서 희석될 수 있기 때문에
    
- 합성곱은 커널이 겹치는 부분이 있지만 풀링은 겹치지 않고 이동

- 최근에는 신경망이 점점 깊어지면서 미세하지만 중요한 특징들이 소실되는 현상을 막기 위해서 풀링층 사용을 줄이는 방식도 도입되고 있음
    - 하지만 여전히 풀링은 적은 계산량으로 좋은 성능을 유지하는 데에 유용한 신경망 계층임

In [3]:
# 풀링
keras.layers.MaxPool2D(2, strides = 2, padding = "valid")

- MaxPool2D의 첫 번째 매개변수는 풀링의 크기
    - 일반적으로 2를 사용(가로세로 크기를 절반으로 줄임)
    - 가로세로 방향의 풀링 크기를 다르게 하려면 튜플로 입력가능하지만 실제로 사용되는 경우는 매우 드문 편
    
- strides : 자동으로 풀링의 크기가 입력되기 때문에 따로 지정할 필요는 없음
- padding : 기본값은 valid. 보통은 풀링에서 패딩을 하지 않기 때문에 따로 지정하는 경우는 거의 없음

# 합성곱 신경망의 구조

<img src = "./image/cnn구조.png">

- 합성곱 신경망은 너비와 높이는 점점 줄어들고 깊이는 점점 깊어지는 것이 특징
- 마지막 분류 신경망에서 특성맵을 모두 펼쳐서 밀집층의 입력으로 사용

- 합성곱 신경망에서 필터는 이미지에 있는 어떤 특징을 찾는 역할
    - 필터의 개수를 늘릴수록, 층이 깊어질수록 데이터의 구체적인 특징을 감지
    - 어떤 특징이 이미지의 어느 위치에 놓이더라도 쉽게 감지할 수 있도록 너비와 높이를 압축(풀링)

# 컬러 이미지를 사용한 합성곱

<img src = "./image/conv_channel.jpg" width = 500 height = 500>

- 흑백 이미지는 2차원 배열로 표현할 수 있음
- 컬러 이미지는 RGB(빨강, 초록, 파랑) 채널로 구성되어 있기 때문에 하나의 이미지를 3차원 배열로 표시

- 깊이가 있는 입력에서 합성곱을 수행하기 위해서는 커널도 깊이가 있어야 함(3차원으로 구성되어야 함)
    - 커널 배열의 깊이는 항상 입력의 깊이와 같음
    
- 입력이나 필터의 차원이 몇 개인지와 관계없이 항상 출력은 하나의 값
- 케라스의 합성곱층은 기본적으로 3차원 입력에 맞춰져있음
    - 흑백 이미지는 깊이 차원이 1인 3차원 배열로 변환하여 전달

# 실습
- 캐글(Kaggle)의 개 고양이 이미지 분류 대회의 이미지 데이터셋을 이용한 CNN 이미지 분류
    - https://www.kaggle.com/c/dogs-vs-cats

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")
# 실행에 필요한 메시지 외 경고가 출력되지 않게 해줌
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from tensorflow import keras

from sklearn.model_selection import train_test_split

## 전처리

In [ ]:
imageDataGenerator = ImageDataGenerator(
    rescale=1./255,                  # 고정 이미지 - 픽셀값을 0~255에서 0~1범위로 변경
    shear_range=0.2,                 # 기울기 범위
    zoom_range=0.2,                  # 확대 범위
    horizontal_flip=True,           # 상하반전
    validation_split=0.1            # 배치비율(예: 0.3, 0.2, 0.25 등등)
)
imageDataGenerator_test = ImageDataGenerator(rescale=1./255)

train_dataset = imageDataGenerator.flow_from_directory(
    "./data/train/",
    target_size=(128,128),
    batch_size=100,
    subset="training",
    class_mode="binary"
)

val_dataset = imageDataGenerator.flow_from_directory(
    "./data/train/",
    target_size=(128,128),
    batch_size=100,
    subset="validation",
    class_mode="binary"
)

test_dataset = imageDataGenerator_test.flow_from_directory(
    "./data/test/",
    target_size=(128,128),
    batch_size=100,
    shuffle=False
)

- 개 이미지 12,500개, 고양이 이미지 12,500개를 train/validation/test 용으로 분리하고, tensorflow의 ImageDataGenerator를 사용할 것이기 때문에 각 라벨(cat,dog)를 폴더별로 따로 분리

## 모델 생성 및 평가

In [128]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 16, 16, 32)     

In [129]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-3model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [130]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [131]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("model.h5")

Epoch 1/100
225/225 [==============================] - 201s 885ms/step - loss: 0.6478 - accuracy: 0.6076 - val_loss: 0.5838 - val_accuracy: 0.6764
Epoch 2/100
225/225 [==============================] - 186s 826ms/step - loss: 0.5646 - accuracy: 0.7076 - val_loss: 0.5241 - val_accuracy: 0.7272
Epoch 3/100
225/225 [==============================] - 185s 822ms/step - loss: 0.5202 - accuracy: 0.7446 - val_loss: 0.4835 - val_accuracy: 0.7792
Epoch 4/100
225/225 [==============================] - 185s 824ms/step - loss: 0.4873 - accuracy: 0.7692 - val_loss: 0.4540 - val_accuracy: 0.7804
Epoch 5/100
225/225 [==============================] - 183s 814ms/step - loss: 0.4716 - accuracy: 0.7815 - val_loss: 0.4525 - val_accuracy: 0.7968
Epoch 6/100
225/225 [==============================] - 186s 826ms/step - loss: 0.4450 - accuracy: 0.7982 - val_loss: 0.4028 - val_accuracy: 0.8128
Epoch 7/100
225/225 [==============================] - 189s 840ms/step - loss: 0.4263 - accuracy: 0.8103 - val_loss: 0

In [132]:
pred = model.predict(test_dataset)

125/125 [==============================] - 37s 291ms/step
